# Nashville Housing Data Cleaning

This dataset revolves around the housing sales in Nashville, Tennessee, and includes several key information such as sale price, sale date, land use, and property details.



In [1]:
import pandas as pd
# Read CSV data into pandas DataFrame
df =pd.read_csv(r'C:\Users\Honar\Downloads\Nashville.csv')
df.head(5)

,UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,OwnerAddress,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath
0,2045,007 00 0 125.00,SINGLE FAMILY,"1808 FOX CHASE DR, GOODLETTSVILLE",09-Apr-13,240000,20130412-0036474,No,"FRAZIER, CYRENTHA LYNETTE","1808 FOX CHASE DR, GOODLETTSVILLE, TN",2.3,GENERAL SERVICES DISTRICT,50000.0,168200.0,235700.0,1986.0,3.0,3.0,0.0
1,16918,007 00 0 130.00,SINGLE FAMILY,"1832 FOX CHASE DR, GOODLETTSVILLE",10-Jun-14,366000,20140619-0053768,No,"BONER, CHARLES & LESLIE","1832 FOX CHASE DR, GOODLETTSVILLE, TN",3.5,GENERAL SERVICES DISTRICT,50000.0,264100.0,319000.0,1998.0,3.0,3.0,2.0
2,54582,007 00 0 138.00,SINGLE FAMILY,"1864 FOX CHASE DR, GOODLETTSVILLE",26-Sep-16,435000,20160927-0101718,No,"WILSON, JAMES E. & JOANNE","1864 FOX CHASE DR, GOODLETTSVILLE, TN",2.9,GENERAL SERVICES DISTRICT,50000.0,216200.0,298000.0,1987.0,4.0,3.0,0.0
3,43070,007 00 0 143.00,SINGLE FAMILY,"1853 FOX CHASE DR, GOODLETTSVILLE",29-Jan-16,255000,20160129-0008913,No,"BAKER, JAY K. & SUSAN E.","1853 FOX CHASE DR, GOODLETTSVILLE, TN",2.6,GENERAL SERVICES DISTRICT,50000.0,147300.0,197300.0,1985.0,3.0,3.0,0.0
4,22714,007 00 0 149.00,SINGLE FAMILY,"1829 FOX CHASE DR, GOODLETTSVILLE",10-Oct-14,278000,20141015-0095255,No,"POST, CHRISTOPHER M. & SAMANTHA C.","1829 FOX CHASE DR, GOODLETTSVILLE, TN",2.0,GENERAL SERVICES DISTRICT,50000.0,152300.0,202300.0,1984.0,4.0,3.0,0.0


### Data Preparation and Cleaning
The insight that we want to derive from the data mainly revolves around sales and land use. Therefore, we need to ensure that the data related to these points are of good quality, and that could be achieved through the process of data cleaning.


In [2]:
"""updates column names for consistency and converts specific columns to numerical types,
    followed by converting  for example 'sale_date' to datetime format."""

# Rename columns
df = df.rename(columns={
    'UniqueID ': 'unique_id',
    'ParcelID': 'parcel_id',
    'LandUse': 'land_use',
    'PropertyAddress': 'property_address',
    'SaleDate': 'sale_date',
    'SalePrice': 'sale_price',
    'LegalReference': 'legal_reference',
    'SoldAsVacant': 'sold_as_vacant',
    'OwnerName': 'owner_name',
    'OwnerAddress': 'owner_address',
    'Acreage': 'acreage',
    'TaxDistrict': 'tax_district',
    'LandValue': 'land_value',
    'BuildingValue': 'building_value',
    'TotalValue': 'total_value',
    'YearBuilt': 'year_built',
    'Bedrooms': 'bedrooms',
    'FullBath': 'full_bath',
    'HalfBath': 'half_bath'
})
# Convert specific columns to numeric
numeric_cols = ['sale_price', 'land_value', 'building_value', 'acreage', 
                'total_value', 'year_built', 'bedrooms', 'full_bath', 'half_bath']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Convert 'sale_date' to datetime if it's not already
df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce', format='%d-%b-%y')

**city and address :**  Extract from property_address

In [3]:
df[['Address', 'city']] = df['property_address'].str.rsplit(', ', n=1, expand=True)
df['city']

0        GOODLETTSVILLE
1        GOODLETTSVILLE
2        GOODLETTSVILLE
3        GOODLETTSVILLE
4        GOODLETTSVILLE
              ...      
56472       NOLENSVILLE
56473       NOLENSVILLE
56474       NOLENSVILLE
56475       NOLENSVILLE
56476       NOLENSVILLE
Name: city, Length: 56477, dtype: object

**sold_as_vacant:** included 'Y', 'Yes' and 'N', 'No'.  
It needs to Replace 'Y' with 'Yes' and 'N' with 'No' in the 'sold_as_vacant' column.

In [4]:
print(" Values Count of sold_as_vacant before replacing:")
print(df['sold_as_vacant'].value_counts())

# Replace 'Y' with 'Yes' and 'N' with 'No' in the 'sold_as_vacant' column
df['sold_as_vacant'].replace({'Y': 'Yes', 'N': 'No'}, inplace=True)

# Use `value_counts()` to verify the updated counts for 'Yes' and 'No' in the 'sold_as_vacant' column.
print(" Values Count of sold_as_vacant After replacing:")
print(df['sold_as_vacant'].value_counts())

 Values Count of sold_as_vacant before replacing:
sold_as_vacant
No     51403
Yes     4623
N        399
Y         52
Name: count, dtype: int64
 Values Count of sold_as_vacant After replacing:
sold_as_vacant
No     51802
Yes     4675
Name: count, dtype: int64



**unique_id:** this is the primary key of this table. We found no duplicates in this column, but we found that there are 3 entries that have missing values for their unique_id. The next action is we are going to remove the entries with missing unique_id.


In [5]:
# Finding Duplicated ID
grouped = df.groupby('unique_id').size()
duplicate_unique_ids = grouped[grouped > 1]
# Find rows where 'UniqueID ' column is null
missing_unique_id = df[df['unique_id'].isnull()]
# Drop rows where 'UniqueID ' column is null
df = df.dropna(subset=['unique_id'])

**land_use:** this is the column that holds the information on land use types across the housing sales. We found several categories that were duplicated or had typos in them, thus we altered the incorrect entries to their respective categories. 
One of the examples is there are entries of “VACANT RESIDENTIAL LAND”, “VACANT RES LAND”, and “VACANT RESIENTIAL LAND”, where the three of them should have been treated as one category, e.g. **“VACANT RESIDENTIAL LAND”**. The data cleaning process had reduced incorrectly labeled categories from 39 to 22.

In [6]:
#This returns several incorrect entries, such as having typo or abbreviated
df['land_use'].unique()

array(['SINGLE FAMILY', 'RESIDENTIAL CONDO', 'VACANT RURAL LAND',
       'VACANT RES LAND', 'DUPLEX', 'VACANT RESIDENTIAL LAND', 'CHURCH',
       'ZERO LOT LINE', 'RESIDENTIAL COMBO/MISC', 'PARSONAGE',
       'OFFICE BLDG (ONE OR TWO STORIES)',
       'GREENBELT/RES\r\nGRRENBELT/RES', 'MOBILE HOME', 'PARKING LOT',
       'CLUB/UNION HALL/LODGE', 'TRIPLEX', 'VACANT ZONED MULTI FAMILY',
       'SPLIT CLASS', 'CONDO', 'QUADPLEX', 'LIGHT MANUFACTURING',
       'FOREST', 'CONVENIENCE MARKET WITHOUT GAS',
       'DORMITORY/BOARDING HOUSE', 'GREENBELT', 'DAY CARE CENTER',
       'METRO OTHER THAN OFC, SCHOOL,HOSP, OR PARK',
       'TERMINAL/DISTRIBUTION WAREHOUSE', 'NIGHTCLUB/LOUNGE',
       'RESTURANT/CAFETERIA', 'VACANT COMMERCIAL LAND',
       'CONDOMINIUM OFC  OR OTHER COM CONDO', 'STRIP SHOPPING CENTER',
       'VACANT RESIENTIAL LAND', 'APARTMENT: LOW RISE (BUILT SINCE 1960)',
       'ONE STORY GENERAL RETAIL STORE', 'NON-PROFIT CHARITABLE SERVICE',
       'SMALL SERVICE SHOP', 'MORTUAR

In [7]:
# Replace 'RESTURANT/CAFETERIA' with 'RESTAURANT' in the 'land_use' column
df['land_use'] = df['land_use'].replace('RESTURANT/CAFETERIA', 'RESTAURANT')

# Replace 'FOREST' and 'GREENBELT/RES' with 'GREENBELT' in the 'land_use' column
df['land_use'] = df['land_use'].replace(['FOREST', 'GREENBELT/RES','GREENBELT/RES\r\nGRRENBELT/RES'], 'GREENBELT')

# Replace categories using the defined mapping
df['land_use'] = df['land_use'].replace(['RESIDENTIAL CONDO','CONDOMINIUM OFC OR OTHER COM CONDO', 'RESIDENTIAL COMBO/MISC'],'CONDOMINIUM')

# Replace each low-rise apartment-related land use category separately
df['land_use'] = df['land_use'].replace(['APARTMENT: LOW RISE (BUILT SINCE 1960)','SPLIT CLASS'], 'LOW RISE APARTMENT')

# Replace each night entertainment-related land use category separately

df['land_use'] = df['land_use'].replace(['NIGHTCLUB/LOUNGE','CLUB/UNION HALL/LODGE'] , 'NIGHT ENTERTAINMENT')

# Replace the typo in the 'land_use' column
df['land_use'] = df['land_use'].replace('OFFICE BLDG (ONE OR TWO STORIES)', 'OFFICE')

# Replace the transportation-related land use category with 'TRANSPORTATION'
df['land_use'] = df['land_use'].replace('METRO OTHER THAN OFC, SCHOOL,HOSP, OR PARK', 'TRANSPORTATION')

# Replace the distribution-related land use category with 'DISTRIBUTION'
df['land_use'] = df['land_use'].replace('TERMINAL/DISTRIBUTION WAREHOUSE', 'DISTRIBUTION')

# Replace the commercial-related land use categories with 'COMMERCIAL LAND'
commercial_land_use = [
    'ONE STORY GENERAL RETAIL STORE',
    'SMALL SERVICE SHOP',
    'CONVENIENCE MARKET WITHOUT GAS',
    'STRIP SHOPPING CENTER',
    'DAY CARE CENTER',
    'LIGHT MANUFACTURING',
    'PARKING LOT',
    'RESTAURANT'
]
df['land_use'] = df['land_use'].replace(commercial_land_use, 'COMMERCIAL LAND')

In [8]:
# land_use columns unique values after cleaning
df['land_use'].unique()

array(['SINGLE FAMILY', 'CONDOMINIUM', 'VACANT RURAL LAND',
       'VACANT RES LAND', 'DUPLEX', 'VACANT RESIDENTIAL LAND', 'CHURCH',
       'ZERO LOT LINE', 'PARSONAGE', 'OFFICE', 'GREENBELT', 'MOBILE HOME',
       'COMMERCIAL LAND', 'NIGHT ENTERTAINMENT', 'TRIPLEX',
       'VACANT ZONED MULTI FAMILY', 'LOW RISE APARTMENT', 'CONDO',
       'QUADPLEX', 'DORMITORY/BOARDING HOUSE', 'TRANSPORTATION',
       'DISTRIBUTION', 'VACANT COMMERCIAL LAND',
       'CONDOMINIUM OFC  OR OTHER COM CONDO', 'VACANT RESIENTIAL LAND',
       'NON-PROFIT CHARITABLE SERVICE', 'MORTUARY/CEMETERY'], dtype=object)

**land_value and building_value:** these are the columns that contain the information on the land and building valuation, and they will be used together to calculate the profit from the sale value. According to our query, it is found nearly 54% of our data had missing land and building values. Thus, we are going to remove the rows that contain null values to ensure our insight quality( the same for city and sale_price)

In [9]:
print(f"Data dimentions before cleaning :{df.shape}")
#  Remove entries with missing land_value and building_value
df = df.dropna(subset=['land_value', 'building_value','city','sale_price'])

print(f"Data dimentions after cleaning :{df.shape}")

Data dimentions before cleaning :(56477, 21)
Data dimentions after cleaning :(25991, 21)


### Data Analysis and Processing
The first analysis will be the answer to the question of ‘How is the business condition in terms of profit growth in the last few years?’ and it will be achieved through deducing land value and building value from the sale value.

In [10]:
# Drop rows with NaN values in 'sale_price', 'land_value', or 'building_value'
df.dropna(subset=['sale_price', 'land_value', 'building_value'], inplace=True)

# Calculate profit for each sale
df['profit'] = df['sale_price'] - df['land_value'] - df['building_value']

# Extract year from sale_date
df['year'] = df['sale_date'].dt.year

# Group by year and calculate total profit
profit_by_year = df.groupby('year')['profit'].sum().reset_index()

# Sort by year in ascending order
profit_by_year = profit_by_year.sort_values(by='year')
print(profit_by_year)

   year       profit
0  2013  -62673914.0
1  2014  173295933.0
2  2015  535178522.0
3  2016  676940295.0
4  2019      29000.0


However, it can be observed that there is no data available for the years 2017 to 2018. Further investigation is needed to uncover the underlying reasons for this absence of data during that specific time period. Possible explanations may include business inactivity, data storage issues, or other factors. Nevertheless, given the nature of the data source and the scope of this project, our focus will be directed towards analyzing the data from 2013 to 2016.

In [11]:

# Filter out data for the year 2019
df_filtered= df[df['sale_date'].dt.year != 2019].copy()

# Calculate profit for each sale
df_filtered['profit'] = df_filtered['sale_price'] - df_filtered['land_value'] - df_filtered['building_value']

# Group by year and calculate total profit
profit_by_year = df_filtered.groupby(df['sale_date'].dt.year)['profit'].sum().reset_index()

# Sort by year in descending order
profit_by_year = profit_by_year.sort_values(by='sale_date', ascending=False)

# Rename columns to match the SQL query
profit_by_year.rename(columns={'sale_date': 'Year', 'profit': 'Profit'}, inplace=True)

# Print the result
print(profit_by_year)


   Year       Profit
3  2016  676940295.0
2  2015  535178522.0
1  2014  173295933.0
0  2013  -62673914.0


The second analysis is to find the most suitable type of land use to be the business focus. To address this question, we will retrieve the number of each type of land use for each year, and then compare the numbers.

In [12]:
"""This code filters out the data for the year 2019, groups the data by year and land_use,
    calculates the count of each land_use type, renames the columns to match the SQL query,
    and sorts the results by year in descending order. Finally, it print#s the result."""

# Group by year and land_use, then calculate the count of each land_use type
count_per_land_use = df_filtered.groupby([df_filtered['sale_date'].dt.year, 'land_use']).size().reset_index(name='Land Use Rate')

# Rename columns 
count_per_land_use.rename(columns={'sale_date': 'Year', 'land_use': 'Land Use Type'}, inplace=True)

# Sort by year in descending order
count_per_land_use = count_per_land_use.sort_values(by='Land Use Rate', ascending=False)
count_per_land_use
# Print the result

,Year,Land Use Type,Land Use Rate
47,2015,SINGLE FAMILY,6273
66,2016,SINGLE FAMILY,5485
29,2014,SINGLE FAMILY,5479
10,2013,SINGLE FAMILY,4617
50,2015,VACANT RESIDENTIAL LAND,498
...,...,...,...
48,2015,TRANSPORTATION,1
25,2014,NIGHT ENTERTAINMENT,1
54,2016,COMMERCIAL LAND,1
26,2014,OFFICE,1


In addition to comparing the rate of the land use types, we also need to consider it from a sales perspective. Here, we will analyze it based on the total revenue generated by each of the land uses.

In [13]:
""""This code will filter out the data for the year 2019, group the data by year and land_use, 
    calculate the sum of sale_price for each land_use type, rename the columns to match the SQL query, 
    sort the results by year in descending order, and finally print the result."""

# Group by year and land_use, then calculate the sum of sale_price for each land_use type
revenue_per_land_use = df_filtered.groupby([df_filtered['sale_date'].dt.year, 'land_use'])['sale_price'].sum().reset_index(name='Revenue per Land Use')

# Rename columns to match the SQL query
revenue_per_land_use.rename(columns={'sale_date': 'Year', 'land_use': 'Land Use Type'}, inplace=True)

# Sort by year in descending order
revenue_per_land_use = revenue_per_land_use.sort_values(by='Year', ascending=False)
# Print the result
print(revenue_per_land_use)

    Year              Land Use Type  Revenue per Land Use
71  2016              ZERO LOT LINE            31904793.0
62  2016        NIGHT ENTERTAINMENT              230000.0
53  2016                     CHURCH             4905000.0
54  2016            COMMERCIAL LAND              375000.0
56  2016   DORMITORY/BOARDING HOUSE             1858000.0
..   ...                        ...                   ...
13  2013            VACANT RES LAND            60515479.0
14  2013          VACANT RURAL LAND              250000.0
15  2013  VACANT ZONED MULTI FAMILY               24000.0
16  2013              ZERO LOT LINE            15386357.0
0   2013                     CHURCH              715000.0

[72 rows x 3 columns]


In [14]:
# Save cleaned data to a new CSV file

cleaned_file_path ='Cleaned_Nashville.csv'
df.to_csv(cleaned_file_path, index=False)